# Shelter Animal Outcomes 7

## K Nearest Neighbors

In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import cross_validation
from sklearn.feature_selection import SelectKBest, chi2
#from sklearn.pipeline import make_pipeline
import pandas as pd

In [2]:
df_train = pd.read_csv('../Shelter_train.csv')
df_test = pd.read_csv('../Shelter_test.csv')

In [3]:
X = df_train.ix[:, :-1]
y = df_train.ix[:, -1]
df_test = df_test.drop('ID', 1)

In [4]:
clf = KNeighborsClassifier()
cross_validation.cross_val_score(clf, X, y, scoring="log_loss")

array([-5.68140971, -5.60506317, -5.79990957])

In [6]:
X_new = SelectKBest(chi2, k=3).fit_transform(X, y)
cross_validation.cross_val_score(clf, X_new, y, scoring="log_loss")
X_test = SelectKBest(chi2, k=3).fit(X, y).transform(df_test)

array([-5.35670728, -6.26972317, -4.560658  ])

In [10]:
#selector = SelectKBest(chi2, k=3)
#predictor = make_pipeline(selector, clf)
#predictor.fit(X, y)
#predictions = predictor.predict_proba(df_test)

#X_test = SelectKBest(chi2, k=3).fit(X, y).transform(df_test)
clf = clf.fit(X_new, y)
predictions = clf.predict_proba(X_test)
output = pd.DataFrame(predictions, columns=['Adoption', 'Died', 'Euthanasia', 'Return_to_owner', 'Transfer'])
output.index.names = ['ID']
output.index += 1
output.head()

,Adoption,Died,Euthanasia,Return_to_owner,Transfer
1,0.4,0.0,0.2,0.0,0.4
2,0.8,0.0,0.0,0.0,0.2
3,0.6,0.0,0.0,0.2,0.2
4,0.0,0.0,0.0,0.0,1.0
5,0.6,0.0,0.0,0.0,0.4


In [16]:
output.to_csv('../submission-KNN.2.0.csv', index_label = 'ID')